# Proyecto

Autor: Fabian Morera Gutierrez

Profesor: Dr. Juan Manuel Esquivel Rodriguez

Curso: Big Data

# Entrenamiento de dos modelos de clasificación binaria para una variable de crítica especializada para un set de VideoJuegos.

El presente programa tiene como finalidad la creación de dos modelos de clasificaciónbinaria, de principio a fin, utilizando el framework Apache Spark. Para tal fin, se utilizará un set de datos con información de ventas y crítica de videojuegos a nivel mundial.

Se asume que dicho conjunto de datos fue previamente depurado, limpiado y preparado para su futura utilización como set de entrenamiento y testeo de un modelo.

## Dependencias
Importamos dependencias requeridas a lo largo del programa aquí.

In [ ]:
# Numpy y Matplotlib.
import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

# Others.
import findspark
import tempfile
import random

# Pyspark main.
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Pyspark machine learning.
from pyspark.ml.classification import LogisticRegression
from pyspark.mllib.regression import LabeledPoint
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.linalg import Vectors
from pyspark.ml.tuning import CrossValidatorModel, CrossValidator, ParamGridBuilder
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler, HashingTF, Tokenizer, StandardScaler
from pyspark.ml import Pipeline

from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.util import MLUtils

# Seaborn.
import seaborn as sns

print("Dependencies loaded.")

# Inspección de datos
Previo a entrenar el modelo es común que se realice algún tipo de descripción de los datos, para tener una idea del tipo de problema con el que nos enfrentamos. A continuación, algunas observaciones interesantes.

In [ ]:
findspark.init('/usr/lib/python3.7/site-packages/pyspark')

# Cargar el conjunto de datos completo. Este paso no realiza ningún ajuste; simplemente lectura
spark = SparkSession \
    .builder \
    .appName("Basic JDBC pipeline") \
    .config("spark.driver.extraClassPath", "postgresql-42.2.14.jar") \
    .config("spark.executor.extraClassPath", "postgresql-42.2.14.jar") \
    .getOrCreate()

# Cargar el conjunto de datos desde la base de datos (Tabla vg_critic_sales).
videogames_df = spark \
    .read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://host.docker.internal:5433/postgres") \
    .option("user", "postgres") \
    .option("password", "testPassword") \
    .option("dbtable", "vg_critic_sales") \
    .load()

print("Data loaded!")
videogames_df.show()

## Información Descriptiva de los Datos.
Se muestran histogramas y resúmenes descriptivos de algunas de las variables disponibles.

In [ ]:
# Información descriptiva de alguno valores relevantes del dataframe.
videogames_df.describe([
    'metascore', 
    'metacritic_user_score', 
    'total_shipped']).show()

## Histogramas.
Creamos algunos histogramas de relevancia, para visualizar la distribución de datos.

In [ ]:
# Creación de histogramas.
def print_hist(rdd_histogram_data):
    heights = np.array(rdd_histogram_data[1])
    full_bins = rdd_histogram_data[0]
    mid_point_bins = full_bins[:-1]
    widths = [abs(i - j) for i, j in zip(full_bins[:-1], full_bins[1:])]
    plt.bar(mid_point_bins, heights, width=widths, color='b')
    plt.show()

# Obtener histograma, con los parámetros de rango determinados
# y para la propiedad especificada.
def create_histogram(h_start=0, h_stop=100, h_step=5, h_prop):
    """
    Params:
        h_start: Histogram start value.
        h_stop: Histogram end value.
        h_step: Iteration size.
        h_prop: Name of property to create Histogram for.
    """
    buckets = np.arange(
        start=h_start, 
        stop=h_stop, 
        step=h_step).tolist()

    rdd_histogram_data = videogames_df\
        .select(h_prop)\
        .rdd\
        .flatMap(lambda x: x)\
        .histogram(buckets)

    print_hist(rdd_histogram_data)

# Metascore Histogram.
create_histogram(0,100,5,'metascore')

# Metacritic user scores histogram.
create_histogram(0,10,0.5,'metacritic_user_score')

# Total Shipped in Millions Histogram.
create_histogram(0,50,5,'total_shipped')

# Critic Positive in base 10 Histogram.
create_histogram(0,100,10,'c_pos')

# Critic Neutral in base 10 Histogram.
create_histogram(0,100,10,'c_neu')

# Critic Negative in base 10 Histogram.
create_histogram(0,100,10,'c_neg')

## Preparación de datos (Vectorización de features)
Para realizar operaciones más detalladas es necesario expresar las filas originales en vectores

In [ ]:
videogames_df = videogames_df \
    .drop('formatted_name')
    .withColumn('label', 
        F.when(F.col("metascore") > 70, 1)
        .otherwise(0))

print("Videogames clean Dataset before vectorization.\n")
videogames_df.show()
videogames_df.printSchema()

# Para realizar operaciones más detalladas es necesario expresar las filas originales en vectores
assembler = VectorAssembler(
    inputCols=[
        'critic_score', 
        'user_score', 
        'total_shipped',
        'year',
        'formatted_name',
        'c_pos',
        'c_neu',
        'c_neg',
        'metascore',
        'u_pos',
        'u_neu',
        'u_neg',
        'metacritic_user_score',
        'isPCPlatform',
        'isX360Platform',
        'isOtherPlatform',
        'isERating',
        'isTRating',
        'isMRating',
        'isOtherRating',
        'isActionGenre',
        'isActionAdventureGenre',
        'isOtherGenre',
        'isOnePlayerOnly',
        'hasOnlineMultiplayer'],
    outputCol='features')

vector_df = assembler.transform(videogames_df)
vector_df = vector_df.select(['features', 'videogames_df'])
vector_df.show()
vector_df.printSchema()

In [ ]:
# Con la representación de vectores podemos calcular correlaciones
pearson_matrix = Correlation.corr(vector_df, 'features').collect()[0][0]

sns.heatmap(pearson_matrix.toArray(), annot=True, fmt=".2f", cmap='viridis')

## Estandarización.

Como recordamos de los módulos anteriores es deseable que los datos se encuentren estandarizados o normalizados, para evitar que la magnitud de ciertos atributos dominen el proceso de entrenamiento. 

In [ ]:
standard_scaler = StandardScaler(inputCol='features', outputCol='scaled')
scale_model = standard_scaler.fit(vector_df)

scaled_df = scale_model.transform(vector_df)
scaled_df = scaled_df.drop("features")
scaled_df = scaled_df.withColumnRenamed("scaled", "features")
scaled_df.show()
scaled_df.printSchema()

## Clasificación Binaria 
Entrenamos dos modelos con técnicas diferentes y evaluamos los resultados.

In [ ]:
# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = scaled_df.randomSplit([0.7, 0.3])

print("Training Data Count: ", trainingData.count())
trainingData.show()
print("Test Data Count:", testData.count())
testData.show()